In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os

print("TensorFlow Version:", tf.__version__)


TensorFlow Version: 1.15.5


In [2]:
cifar_dir = os.path.expanduser("~/JPTR_NTBK/CIFAR_10_DATA/cifar-10-batches-py/")

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

def load_cifar_data():
    X_train, y_train = [], []

    for i in range(1, 6):  
        data_batch = unpickle(os.path.join(cifar_dir, f"data_batch_{i}"))
        X_train.append(data_batch[b'data'])
        y_train.append(data_batch[b'labels'])

    X_train = np.concatenate(X_train, axis=0).reshape(-1, 32, 32, 3)
    y_train = np.concatenate(y_train, axis=0)

    
    test_batch = unpickle(os.path.join(cifar_dir, "test_batch"))
    X_test = test_batch[b'data'].reshape(-1, 32, 32, 3)
    y_test = np.array(test_batch[b'labels'])

    return (X_train, y_train), (X_test, y_test)


(X_train, y_train), (X_test, y_test) = load_cifar_data()

print("Training set shape:", X_train.shape, "Labels:", y_train.shape)
print("Test set shape:", X_test.shape, "Labels:", y_test.shape)

Training set shape: (50000, 32, 32, 3) Labels: (50000,)
Test set shape: (10000, 32, 32, 3) Labels: (10000,)


In [3]:
X_train, X_test = X_train / 255.0, X_test / 255.0


In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
y = tf.placeholder(tf.int32, [None], name="y")

conv1 = tf.layers.conv2d(X, filters=32, kernel_size=3, padding='same', activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2)

conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=3, padding='same', activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, pool_size=2, strides=2)

flat = tf.layers.flatten(pool2)

fc1 = tf.layers.dense(flat, units=128, activation=tf.nn.relu)

logits = tf.layers.dense(fc1, units=10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.cast(y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))



Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)  
    print("Model initialized")


Model initialized


In [8]:
num_epochs = 5000  
batch_size = 64     

with tf.Session() as sess:
    sess.run(init)  

    for epoch in range(num_epochs):
        
        indices = np.random.choice(len(X_train), batch_size)
        X_batch, y_batch = X_train[indices], y_train[indices]

        _, loss_value = sess.run([optimizer, loss], feed_dict={X: X_batch, y: y_batch}
        if epoch % 100 == 0:
            acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
            print(f"Epoch {epoch}: Loss = {loss_value:.4f}, Test Accuracy = {acc:.4%}")

    print("Training complete!")
    
    final_acc = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
    print(f"Final Test Accuracy: {final_acc:.4%}")


ERROR! Session/line number was not unique in database. History logging moved to new session 7
Epoch 0: Loss = 2.3379, Test Accuracy = 10.0000%
Epoch 100: Loss = 1.8263, Test Accuracy = 33.5300%
Epoch 200: Loss = 1.5205, Test Accuracy = 42.0500%
Epoch 300: Loss = 1.4317, Test Accuracy = 45.8200%
Epoch 400: Loss = 1.3529, Test Accuracy = 45.7300%
Epoch 500: Loss = 1.1013, Test Accuracy = 49.2300%
Epoch 600: Loss = 1.6794, Test Accuracy = 50.1200%
Epoch 700: Loss = 1.5378, Test Accuracy = 51.1200%
Epoch 800: Loss = 1.1003, Test Accuracy = 51.4300%
Epoch 900: Loss = 1.2741, Test Accuracy = 55.4600%
Epoch 1000: Loss = 1.4220, Test Accuracy = 54.2600%
Epoch 1100: Loss = 1.3912, Test Accuracy = 55.0200%
Epoch 1200: Loss = 1.1747, Test Accuracy = 56.7000%
Epoch 1300: Loss = 1.1948, Test Accuracy = 57.1500%
Epoch 1400: Loss = 1.0090, Test Accuracy = 57.5800%
Epoch 1500: Loss = 1.1698, Test Accuracy = 56.6500%
Epoch 1600: Loss = 1.1001, Test Accuracy = 58.2400%
Epoch 1700: Loss = 0.9210, Test Ac